In [1]:
fin = open('example2.txt')

In [41]:
# save the baskets in T
fin.seek(0)
transactions=[]
for x in fin:
    X = x.strip()
    Y = X.split(',')
    transactions.append(Y)
fin.close()

ValueError: I/O operation on closed file.

In [3]:
transactions

[['A', 'C', 'D'], ['B', 'C', 'E'], ['A', 'B', 'C', 'E'], ['B', 'E']]

In [4]:
def get_unique_items(transaction_list):
    unique_items = []
    for transaction in transaction_list:
        for item in transaction:
            if item not in unique_items:
                unique_items.append(item)
    unique_items.sort()
    return unique_items

In [6]:
unique_items = get_unique_items(transactions)
unique_items

['A', 'B', 'C', 'D', 'E']

In [20]:
def generate_item_frequencies(unique_items, transaction_list):
    frequencies = {}
    for item in unique_items:
        count = 0
        for transaction in transaction_list:
            if item in transaction:
                count += 1
        frequencies[item] = count
    return frequencies
        

In [25]:
freqs = generate_item_frequencies(unique_items,transactions)
freqs

{'A': 2, 'B': 3, 'C': 3, 'D': 1, 'E': 3}

In [22]:
# assume min support threshold is 2; 2/4 = .5
min_support = 2


In [26]:
def generate_L1(frequencies, min_support):
    L1 = {}
    for k in frequencies.keys():
        if frequencies[k] >= min_support:
            L1[k] = frequencies[k]
    return L1

In [27]:
L1 = generate_L1(freqs, min_support)

In [28]:
L1

{'A': 2, 'B': 3, 'C': 3, 'E': 3}

In [29]:
def generate_C2_pairs(L1):
    pairs = []
    keys = list(L1.keys())
    for i in range(len(keys) - 1):
        for j in range( i + 1, len(keys)):
            pair = [keys[i], keys[j]]
            pair.sort()
            pairs.append(pair)
    return pairs

In [30]:
C2_pairs = generate_C2_pairs(L1)
C2_pairs

[['A', 'B'], ['A', 'C'], ['A', 'E'], ['B', 'C'], ['B', 'E'], ['C', 'E']]

In [31]:
def included(transaction, pair):
    subset = True
    for x in pair:
        if x not in transaction:
            subset = False
    return subset

In [32]:
def generate_C2(pairs, transaction_list):
    d = {}
    for pair in pairs:
        count = 0
        for transaction in transaction_list:
            if included(transaction,pair):
                count += 1
        d[tuple(pair)] = count
    return d

In [33]:
C2 = generate_C2(C2_pairs, transactions)
C2

{('A', 'B'): 1,
 ('A', 'C'): 2,
 ('A', 'E'): 1,
 ('B', 'C'): 2,
 ('B', 'E'): 3,
 ('C', 'E'): 2}

In [34]:
# filter out item sets that dont satisfy min support threshold

def generate_L2(C2, min_support):
    d = C2
    L2 = {}
    for k in d.keys():
        if d[k] >=  min_support:
            L2[k] = d[k]
    return L2

In [35]:
L2 = generate_L2(C2,min_support)
L2

{('A', 'C'): 2, ('B', 'C'): 2, ('B', 'E'): 3, ('C', 'E'): 2}

In [36]:
L1_1 = {}
for x in L1.keys():
    value = L1[x]
    t_i = tuple(x)
    L1_1[t_i] = value

In [37]:
L1_1

{('A',): 2, ('B',): 3, ('C',): 3, ('E',): 3}

In [39]:
# This function is to generate rules from the itemsets of two items.
def generate_L2_rules(L2, L1_1):
    for x in L2.keys():
        print(x)
        print(x[0] + " => " + x[1])
        rule = x[0] + " => " + x[1]
        print(L1_1[tuple(x[0])])
        print(L1_1[tuple(x[1])])
        print("%s ( %d/%d ) %.2f "%(rule, L2[x],L1_1[tuple(x[0])], float(L2[x])/float(L1_1[tuple(x[0])]) ))
        print(round(float(L2[x])/float(L1_1[tuple(x[0])])), 2)
        print("---")
    for x in L2.keys():
        print(x)
        print(x[1] + " => " + x[0])
        rule = x[1] + " => " + x[0]
        print(L1_1[tuple(x[0])])
        print(L1_1[tuple(x[1])])
        print("%s ( %d/%d ) %.2f "%(rule, L2[x],L1_1[tuple(x[1])],float(L2[x])/float(L1_1[tuple(x[1])])  ))
        print(float(L2[x])/float(L1_1[tuple(x[1])]))
        print("---")

In [40]:
generate_L2_rules(L2,L1_1)

('A', 'C')
A => C
2
3
A => C ( 2/2 ) 1.00 
1 2
---
('B', 'C')
B => C
3
3
B => C ( 2/3 ) 0.67 
1 2
---
('B', 'E')
B => E
3
3
B => E ( 3/3 ) 1.00 
1 2
---
('C', 'E')
C => E
3
3
C => E ( 2/3 ) 0.67 
1 2
---
('A', 'C')
C => A
2
3
C => A ( 2/3 ) 0.67 
0.6666666666666666
---
('B', 'C')
C => B
3
3
C => B ( 2/3 ) 0.67 
0.6666666666666666
---
('B', 'E')
E => B
3
3
E => B ( 3/3 ) 1.00 
1.0
---
('C', 'E')
E => C
3
3
E => C ( 2/3 ) 0.67 
0.6666666666666666
---
